# SSYS Wheel Strategy - Share Accumulation

**Goal:** Accumulate 100 shares of SSYS (Stratasys) using the wheel strategy, then generate income via covered calls.

## Why SSYS for the Wheel?
- **Low capital requirement:** ~$1,000 for 100 shares
- **Active options market:** 88k+ call open interest
- **Thesis:** 3D printing + AI manufacturing buildout
- **Good premiums:** ~3-6% per month relative to capital

## The Wheel Strategy
```
PHASE 1: Sell Cash-Secured Puts
         → Collect premium
         → If assigned: Own shares at discount
         → If not: Keep premium, repeat

PHASE 2: Sell Covered Calls (once you own 100 shares)
         → Collect premium
         → If called away: Sell at profit, restart Phase 1
         → If not: Keep shares + premium, repeat
```

In [ ]:
pip install yf

In [ ]:
# Imports
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dataclasses import dataclass
from typing import Optional
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

ModuleNotFoundError: No module named 'yfinance'

## Position Tracker

Track your wheel trades and calculate running cost basis.

In [ ]:
@dataclass
class WheelTrade:
    """Record of a single wheel trade."""
    date: str
    trade_type: str  # 'sell_put', 'put_assigned', 'sell_call', 'call_assigned'
    strike: float
    premium: float
    expiration: str
    notes: str = ""
    
class WheelTracker:
    """Track wheel strategy positions and calculate cost basis."""
    
    def __init__(self, symbol: str):
        self.symbol = symbol
        self.trades: list[WheelTrade] = []
        self.shares_owned = 0
        self.cost_basis = 0.0
        self.total_premium_collected = 0.0
        self.cash_reserved = 0.0
        self.active_put: Optional[WheelTrade] = None
        self.active_call: Optional[WheelTrade] = None
        
    def sell_put(self, strike: float, premium: float, expiration: str, notes: str = ""):
        """Record selling a cash-secured put."""
        trade = WheelTrade(
            date=datetime.now().strftime('%Y-%m-%d'),
            trade_type='sell_put',
            strike=strike,
            premium=premium,
            expiration=expiration,
            notes=notes
        )
        self.trades.append(trade)
        self.total_premium_collected += premium * 100
        self.cash_reserved = strike * 100
        self.active_put = trade
        print(f"SOLD PUT: {self.symbol} ${strike} Put @ ${premium:.2f}")
        print(f"  Premium collected: ${premium * 100:.0f}")
        print(f"  Cash reserved: ${self.cash_reserved:,.0f}")
        print(f"  Effective buy price if assigned: ${strike - premium:.2f}")
        
    def put_assigned(self):
        """Record put assignment - you now own shares."""
        if not self.active_put:
            print("No active put to assign!")
            return
            
        trade = WheelTrade(
            date=datetime.now().strftime('%Y-%m-%d'),
            trade_type='put_assigned',
            strike=self.active_put.strike,
            premium=0,
            expiration=self.active_put.expiration,
            notes=f"Assigned from {self.active_put.strike} put"
        )
        self.trades.append(trade)
        self.shares_owned = 100
        self.cost_basis = self.active_put.strike - self.active_put.premium
        self.cash_reserved = 0
        self.active_put = None
        print(f"PUT ASSIGNED: Now own 100 shares of {self.symbol}")
        print(f"  Cost basis: ${self.cost_basis:.2f}/share")
        
    def put_expired(self):
        """Record put expiring worthless."""
        if not self.active_put:
            print("No active put!")
            return
            
        print(f"PUT EXPIRED WORTHLESS: Kept ${self.active_put.premium * 100:.0f} premium")
        self.cash_reserved = 0
        self.active_put = None
        
    def sell_call(self, strike: float, premium: float, expiration: str, notes: str = ""):
        """Record selling a covered call."""
        if self.shares_owned < 100:
            print("Need 100 shares to sell covered call!")
            return
            
        trade = WheelTrade(
            date=datetime.now().strftime('%Y-%m-%d'),
            trade_type='sell_call',
            strike=strike,
            premium=premium,
            expiration=expiration,
            notes=notes
        )
        self.trades.append(trade)
        self.total_premium_collected += premium * 100
        self.cost_basis -= premium  # Lower cost basis
        self.active_call = trade
        print(f"SOLD CALL: {self.symbol} ${strike} Call @ ${premium:.2f}")
        print(f"  Premium collected: ${premium * 100:.0f}")
        print(f"  New cost basis: ${self.cost_basis:.2f}/share")
        print(f"  Max profit if called: ${(strike - self.cost_basis) * 100:.0f}")
        
    def call_assigned(self):
        """Record call assignment - shares called away."""
        if not self.active_call:
            print("No active call!")
            return
            
        profit = (self.active_call.strike - self.cost_basis) * 100
        print(f"CALL ASSIGNED: Sold 100 shares at ${self.active_call.strike}")
        print(f"  Profit: ${profit:.0f}")
        print(f"  Wheel complete! Ready to restart with puts.")
        
        trade = WheelTrade(
            date=datetime.now().strftime('%Y-%m-%d'),
            trade_type='call_assigned',
            strike=self.active_call.strike,
            premium=0,
            expiration=self.active_call.expiration,
            notes=f"Called away at {self.active_call.strike}, profit ${profit:.0f}"
        )
        self.trades.append(trade)
        self.shares_owned = 0
        self.cost_basis = 0
        self.active_call = None
        
    def call_expired(self):
        """Record call expiring worthless."""
        if not self.active_call:
            print("No active call!")
            return
            
        print(f"CALL EXPIRED: Kept shares + ${self.active_call.premium * 100:.0f} premium")
        print(f"  Cost basis: ${self.cost_basis:.2f}/share")
        self.active_call = None
        
    def status(self):
        """Print current position status."""
        ticker = yf.Ticker(self.symbol)
        current_price = ticker.history(period='1d')['Close'].iloc[-1]
        
        print(f"\n{'='*60}")
        print(f"{self.symbol} WHEEL STATUS")
        print(f"{'='*60}")
        print(f"Current Price: ${current_price:.2f}")
        print(f"\nPOSITION:")
        print(f"  Shares Owned: {self.shares_owned}")
        if self.shares_owned > 0:
            print(f"  Cost Basis: ${self.cost_basis:.2f}/share")
            unrealized = (current_price - self.cost_basis) * self.shares_owned
            print(f"  Unrealized P/L: ${unrealized:,.0f} ({(current_price/self.cost_basis - 1)*100:+.1f}%)")
        print(f"\nPREMIUM COLLECTED:")
        print(f"  Total: ${self.total_premium_collected:,.0f}")
        print(f"\nACTIVE OPTIONS:")
        if self.active_put:
            print(f"  Short Put: ${self.active_put.strike} exp {self.active_put.expiration}")
        if self.active_call:
            print(f"  Short Call: ${self.active_call.strike} exp {self.active_call.expiration}")
        if not self.active_put and not self.active_call:
            print(f"  None - ready for new trade")
        print()
        
    def trade_history(self):
        """Show all trades."""
        if not self.trades:
            print("No trades yet.")
            return
            
        print(f"\n{'='*60}")
        print(f"TRADE HISTORY")
        print(f"{'='*60}")
        for i, t in enumerate(self.trades, 1):
            print(f"{i}. {t.date} | {t.trade_type.upper()} | ${t.strike} | Premium: ${t.premium:.2f}")
            if t.notes:
                print(f"   Note: {t.notes}")

## Initialize SSYS Wheel Tracker

In [ ]:
# Create tracker for SSYS
ssys_wheel = WheelTracker('SSYS')
ssys_wheel.status()

## Current Options Analysis

Find the best puts to sell for accumulation.

In [ ]:
def analyze_wheel_options(symbol: str):
    """Analyze options for wheel strategy."""
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period='5d')
    current_price = hist['Close'].iloc[-1]
    
    print(f"{'='*70}")
    print(f"{symbol} WHEEL OPTIONS ANALYSIS")
    print(f"Current Price: ${current_price:.2f}")
    print(f"{'='*70}\n")
    
    expirations = ticker.options
    today = datetime.now()
    
    # Filter for 30-90 DTE (ideal for wheel)
    good_exps = []
    for exp in expirations:
        exp_date = datetime.strptime(exp, '%Y-%m-%d')
        dte = (exp_date - today).days
        if 25 <= dte <= 120:
            good_exps.append((exp, dte))
    
    # PUTS FOR ACCUMULATION
    print("PUTS TO SELL (for share accumulation):")
    print(f"{'Expiration':<12} {'DTE':<5} {'Strike':<8} {'Bid':<8} {'Eff Cost':<10} {'Discount':<10} {'ROI/Mo':<8}")
    print("-" * 70)
    
    put_recommendations = []
    
    for exp, dte in good_exps[:3]:
        chain = ticker.option_chain(exp)
        puts = chain.puts
        
        for _, row in puts.iterrows():
            strike = row['strike']
            bid = row['bid'] if pd.notna(row['bid']) else 0
            
            # Focus on strikes near or below current price
            if strike < current_price * 0.75 or strike > current_price * 1.05:
                continue
            if bid < 0.10:
                continue
                
            eff_cost = strike - bid
            discount = ((current_price - eff_cost) / current_price) * 100
            monthly_roi = (bid / strike) / (dte / 30) * 100
            
            print(f"{exp:<12} {dte:<5} ${strike:<7.0f} ${bid:<7.2f} ${eff_cost:<9.2f} {discount:>8.1f}%  {monthly_roi:>6.1f}%")
            
            put_recommendations.append({
                'exp': exp, 'dte': dte, 'strike': strike, 
                'bid': bid, 'eff_cost': eff_cost, 'discount': discount
            })
    
    print()
    
    # CALLS FOR INCOME (once you own shares)
    print("CALLS TO SELL (once you own 100 shares):")
    print(f"{'Expiration':<12} {'DTE':<5} {'Strike':<8} {'Bid':<8} {'% Above':<10} {'If Called':<12} {'ROI/Mo':<8}")
    print("-" * 75)
    
    for exp, dte in good_exps[:3]:
        chain = ticker.option_chain(exp)
        calls = chain.calls
        
        for _, row in calls.iterrows():
            strike = row['strike']
            bid = row['bid'] if pd.notna(row['bid']) else 0
            
            # Focus on strikes above current price
            if strike < current_price * 1.0 or strike > current_price * 1.5:
                continue
            if bid < 0.10:
                continue
                
            pct_above = ((strike - current_price) / current_price) * 100
            if_called = (strike - current_price + bid) * 100
            monthly_roi = (bid / current_price) / (dte / 30) * 100
            
            print(f"{exp:<12} {dte:<5} ${strike:<7.0f} ${bid:<7.2f} {pct_above:>8.1f}%   ${if_called:<10.0f}  {monthly_roi:>6.1f}%")
    
    return put_recommendations

# Run analysis
recommendations = analyze_wheel_options('SSYS')

## Record Your First Trade

When you sell a put, record it here:

In [ ]:
# Example: Sell a cash-secured put to start accumulating
# Uncomment and modify when you make the trade:

# ssys_wheel.sell_put(
#     strike=10.0,        # Strike price
#     premium=0.65,       # Premium received per share
#     expiration='2026-03-20',  # Expiration date
#     notes='First wheel trade - accumulation'
# )

## When Put Gets Assigned

Run this when your put is assigned and you own shares:

In [ ]:
# Uncomment when assigned:
# ssys_wheel.put_assigned()

## When Put Expires Worthless

Run this if your put expires and you keep the premium:

In [ ]:
# Uncomment if put expires worthless:
# ssys_wheel.put_expired()

## Sell Covered Call (Once You Own Shares)

After being assigned, sell a covered call:

In [ ]:
# Uncomment when selling covered call:
# ssys_wheel.sell_call(
#     strike=12.0,
#     premium=0.80,
#     expiration='2026-03-20',
#     notes='First covered call'
# )

## Call Outcomes

In [ ]:
# If called away:
# ssys_wheel.call_assigned()

# If call expires worthless:
# ssys_wheel.call_expired()

## Check Position Status

In [ ]:
# Check current status anytime
ssys_wheel.status()

## Trade History

In [ ]:
# View all trades
ssys_wheel.trade_history()

## Momentum Check

Before each trade, check if momentum still supports the thesis.

In [ ]:
def momentum_check(symbol: str):
    """Quick momentum check before trading."""
    ticker = yf.Ticker(symbol)
    hist = ticker.history(period='3mo')
    
    current = hist['Close'].iloc[-1]
    sma_20 = hist['Close'].tail(20).mean()
    sma_50 = hist['Close'].tail(50).mean()
    
    # RSI
    delta = hist['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    rsi_val = rsi.iloc[-1]
    
    # Volume
    avg_vol = hist['Volume'].tail(20).mean()
    current_vol = hist['Volume'].iloc[-1]
    
    print(f"\n{'='*50}")
    print(f"{symbol} MOMENTUM CHECK")
    print(f"{'='*50}")
    print(f"Price: ${current:.2f}")
    print(f"vs 20 SMA: {((current/sma_20)-1)*100:+.1f}%")
    print(f"vs 50 SMA: {((current/sma_50)-1)*100:+.1f}%")
    print(f"RSI(14): {rsi_val:.1f}")
    print(f"Volume: {current_vol/avg_vol:.1f}x average")
    print()
    
    # Score
    score = 0
    if current > sma_20: score += 1
    if current > sma_50: score += 1
    if 40 < rsi_val < 70: score += 1
    if current_vol > avg_vol: score += 1
    
    signals = ['WEAK', 'BELOW AVG', 'NEUTRAL', 'GOOD', 'STRONG']
    print(f"MOMENTUM: {signals[score]} ({score}/4)")
    
    if rsi_val > 70:
        print("WARNING: RSI overbought - consider waiting for pullback")
    if current < sma_20:
        print("WARNING: Below 20 SMA - trend weakening")
        
    return score >= 2

# Run check
is_favorable = momentum_check('SSYS')

## Risk Management

Key rules for the wheel strategy:

In [ ]:
print("""
WHEEL STRATEGY RULES
====================

1. ONLY SELL PUTS ON STOCKS YOU WANT TO OWN
   - If assigned, you're buying at the strike price
   - Make sure you're bullish on the underlying

2. KEEP ENOUGH CASH TO COVER ASSIGNMENT
   - Cash-secured put needs: Strike × 100
   - SSYS $10 put = $1,000 cash reserved

3. CHOOSE STRIKES WISELY
   - For accumulation: ATM or slightly OTM puts
   - For income only: Further OTM puts (less likely assigned)

4. 30-45 DTE IS THE SWEET SPOT
   - Theta decay accelerates
   - Enough time for stock to move

5. DON'T CHASE PREMIUM
   - High premium = high risk
   - 2-5% monthly return is sustainable

6. HAVE AN EXIT PLAN
   - Close at 50% profit if reached early
   - Roll if going against you (or accept assignment)

SSYS-SPECIFIC NOTES:
- Small cap, can be volatile
- Thesis: 3D printing + AI manufacturing
- Watch for earnings surprises
- $1,000 max risk per put contract
""")

## P&L Projection

In [ ]:
def project_wheel_returns(strike: float, put_premium: float, call_premium: float, cycles_per_year: int = 6):
    """Project annual returns from wheel strategy."""
    
    capital = strike * 100  # Cash needed
    
    # Scenario 1: All puts expire worthless (never get shares)
    put_only_annual = put_premium * 100 * cycles_per_year
    put_only_roi = (put_only_annual / capital) * 100
    
    # Scenario 2: Get assigned, then called away each cycle
    full_wheel_per_cycle = (put_premium + call_premium) * 100
    full_wheel_annual = full_wheel_per_cycle * cycles_per_year
    full_wheel_roi = (full_wheel_annual / capital) * 100
    
    print(f"\nWHEEL RETURN PROJECTION")
    print(f"={'='*50}")
    print(f"Strike: ${strike} | Capital: ${capital:,}")
    print(f"Put Premium: ${put_premium:.2f} | Call Premium: ${call_premium:.2f}")
    print(f"Cycles/Year: {cycles_per_year}")
    print()
    print(f"SCENARIO 1 - Puts Only (never assigned):")
    print(f"  Annual Premium: ${put_only_annual:,.0f}")
    print(f"  ROI: {put_only_roi:.1f}%")
    print()
    print(f"SCENARIO 2 - Full Wheel (assigned + called each cycle):")
    print(f"  Annual Premium: ${full_wheel_annual:,.0f}")
    print(f"  ROI: {full_wheel_roi:.1f}%")
    print()
    print(f"REALISTIC (mix of both): ~{(put_only_roi + full_wheel_roi)/2:.0f}% annually")

# Project SSYS wheel returns
project_wheel_returns(strike=10, put_premium=0.50, call_premium=0.60, cycles_per_year=6)

---

## Quick Reference

```python
# Start wheel - sell put
ssys_wheel.sell_put(strike=10, premium=0.50, expiration='2026-03-20')

# If assigned
ssys_wheel.put_assigned()

# If expired worthless
ssys_wheel.put_expired()

# Sell covered call
ssys_wheel.sell_call(strike=12, premium=0.60, expiration='2026-04-17')

# If called away
ssys_wheel.call_assigned()

# If expired worthless
ssys_wheel.call_expired()

# Check status
ssys_wheel.status()
```